# __Treinamento do Modelo de Classificação de Tickets__

Neste notebook realizamos a fase de **modelagem supervisionada** do sistema de classificação de tickets.

Partimos de um dataset pseudo-rotulado, criado a partir da clusterização semântica, e treinamos modelos capazes de prever automaticamente a classe de novos tickets.

O objetivo é construir um modelo que generalize bem, seja estável e possa ser utilizado em produção via API.


In [ ]:
import os
import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

import joblib

import warnings
warnings.filterwarnings("ignore")


##

## **Carregamento do dataset rotulado**

Utilizamos o dataset gerado na fase de análise, no qual cada ticket já possui uma classe associada.

Esse dataset representa a versão inicial supervisionada do sistema e é a base para o treinamento do classificador.


In [25]:
df = pd.read_csv("../data/processed/tickets_labeled.csv")
df.head()

,ticket_id,subject,body,class
0,2,Erro na Autocompletação de Código do IntelliJ ...,"Prezado Suporte ao Cliente <name>,\n\nEstou es...",Cloud & Serviços Digitais
1,28,Problemas intermitentes de exibição após atual...,"Caro Suporte ao Cliente,\n\n Estou entrando em...",Dispositivos Eletrônicos
2,45,Pedido de Assistência para Administração de Se...,"Prezado Suporte ao Cliente de Serviços de TI,\...",Cloud & Serviços Digitais
3,46,NaN,"Prezado <name>,\n\nSentimos muito ao saber sob...",Dispositivos Eletrônicos
4,63,Urgente: Relato de Falha na Bateria do MacBook...,"Prezado Atendimento ao Cliente,<br><br>Recente...",Loja Online e Casos Especiais


##

## __Preparação do texto para modelagem__

Para cada ticket, combinamos assunto e corpo em um único campo de texto e aplicamos a mesma normalização utilizada durante a fase de análise.

Isso garante consistência entre os dados usados no clustering e no treinamento do modelo supervisionado.


In [26]:
df["text"] = df["subject"].fillna("").astype(str) + " " + df["body"].fillna("").astype(str)

##

## **Geração de _embeddings_**

Os textos são convertidos em embeddings semânticos usando o mesmo modelo Sentence-BERT.

Esses vetores capturam o significado dos tickets e servem como entrada para os algoritmos de Machine Learning.


In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")
X = model.encode(df["text"].tolist(), show_progress_bar=True)
y = df["class"].values

Batches: 100%|██████████| 15/15 [00:05<00:00,  2.91it/s]


##

## __Divisão entre dados de treino e teste__

O dataset é dividido em conjuntos de treino e teste, preservando a proporção entre as classes.

Isso permite avaliar a capacidade do modelo de generalizar para tickets nunca vistos.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

##

## __Comparação de diferentes algoritmos__

Serão testados modelos como Regressão Logística, _Random Forest_ e _Gradient Boosting_.

Os modelos são comparados utilizando:
- F1-score médio via cross-validation
- ROC AUC multiclasse

O objetivo é selecionar o modelo que oferece melhor equilíbrio entre performance e generalização.

In [ ]:
def benchmark_models(X_train, X_test, y_train, y_test):
    """
    Treina múltiplos modelos, executa cross-validation
    e compara desempenho por F1 e ROC AUC.

    Retorna:
        DataFrame com métricas
        Modelo vencedor
    """

    models = {
        "LogisticRegression": LogisticRegression(max_iter=3000, class_weight="balanced"),
        "RandomForest": RandomForestClassifier(n_estimators=300, random_state=42, class_weight="balanced"),
        "GradientBoosting": GradientBoostingClassifier(random_state=42),
    }

    results = []

    classes = np.unique(y_train)
    y_test_bin = label_binarize(y_test, classes=classes)

    for name, model in models.items():
        # Fit
        model.fit(X_train, y_train)

        # Cross-validation F1
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring="f1_weighted")

        # Predict probabilities
        y_proba = model.predict_proba(X_test)

        # ROC AUC
        auc = roc_auc_score(y_test_bin, y_proba, average="macro", multi_class="ovr")

        results.append({
            "model": name,
            "cv_f1_mean": cv_scores.mean(),
            "cv_f1_std": cv_scores.std(),
            "roc_auc": auc,
        })

    results_df = pd.DataFrame(results).sort_values("cv_f1_mean", ascending=False)

    # Melhor modelo = maior ROC AUC
    best_model_name = results_df.iloc[0]["model"]
    best_model = models[best_model_name]

    return results_df, best_model


In [30]:
results, best_model = benchmark_models(X_train, X_test, y_train, y_test)
results

,model,cv_f1_mean,cv_f1_std,roc_auc
0,LogisticRegression,0.965856,0.024015,0.998109
1,RandomForest,0.900420,0.022789,0.999716
2,GradientBoosting,0.896886,0.026476,0.999281


##

## __Seleção do modelo final__

O modelo com melhor desempenho global e menor variância é selecionado como classificador oficial do sistema.

Esse modelo será treinado em todo o dataset rotulado e utilizado em produção.

In [31]:
best_model

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :ter

In [32]:
best_model.fit(X_train, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :ter

##

## __Avaliação do modelo__

A qualidade do classificador é avaliada usando métricas de classificação multiclasse, como precisão, recall e F1-score para cada categoria.


In [ ]:
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))


                               precision    recall  f1-score   support

    Cloud & Serviços Digitais       0.98      0.98      0.98        50
     Dispositivos Eletrônicos       0.94      0.94      0.94        17
       Infraestrutura de Rede       1.00      1.00      1.00         9
Loja Online e Casos Especiais       1.00      1.00      1.00        12
        Suporte a Impressoras       1.00      1.00      1.00         7

                     accuracy                           0.98        95
                    macro avg       0.98      0.98      0.98        95
                 weighted avg       0.98      0.98      0.98        95



##

## **Validação cruzada e _Overfitting_**

Utilizamos cross-validation para medir a estabilidade do modelo e identificar possíveis sinais de overfitting, analisando a variação das métricas entre os folds.

In [ ]:
scores = cross_val_score(best_model, X, y, cv=5, scoring="f1_weighted")
scores, scores.mean(), scores.std()

(array([1.        , 0.9895709 , 0.97947631, 0.96857109, 0.94675894]),
 np.float64(0.976875448065601),
 np.float64(0.018322935974224346))

##

## **Treinamento com todo o dataset**

Após a escolha do melhor algoritmo, o modelo é treinado novamente utilizando todos os dados disponíveis, maximizando o uso da informação antes do deploy.


In [35]:
best_model.fit(X, y)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :ter

##

## **Salvamento dos Artefatos**

O classificador treinado e o modelo de embeddings são salvos em disco.

Esses artefatos serão carregados pela API e pelo script de execução local, garantindo consistência entre treino e produção.


In [ ]:
os.makedirs("../models", exist_ok=True)

joblib.dump(best_model, "../models/ticket_classifier.joblib")
joblib.dump(model, "../models/embedding_model.joblib")

print("Modelos salvos em ../models/")


Modelos salvos em ../models/


In [ ]:
# Teste Rápido para se os modelos carregam corretamente
clf_loaded = joblib.load("../models/ticket_classifier.joblib")
emb_loaded = joblib.load("../models/embedding_model.joblib")

sample = df["text"].iloc[0]
vec = emb_loaded.encode([sample])
pred = clf_loaded.predict(vec)

print("Texto:", sample[:120])
print("Classe prevista:", pred[0])


Texto: Erro na Autocompletação de Código do IntelliJ IDEA Prezado Suporte ao Cliente <name>,

Estou escrevendo para trazer à su
Classe prevista: Cloud & Serviços Digitais
